In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
sys.path.append('../saildrone/subroutines/')
import warnings
warnings.simplefilter('ignore') # filter some warning messages
from glob import glob


# read in collocated files

In [3]:
data_dir = 'F:/data/cruise_data/saildrone/sss/sss_collocations/'
data_dir_out = 'F:/data/cruise_data/saildrone/sss/sss_collocations_norepeat/'
filenames = [x for x in glob(data_dir+'*.nc')]
filenames

['F:/data/cruise_data/saildrone/sss/sss_collocations\\PMEL_Arctic_2015_sd126-ALL-1_min-v1jplv04.3_filesave4.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations\\PMEL_Arctic_2015_sd126-ALL-1_min-v1rssv04.0_filesave4.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations\\PMEL_Arctic_2015_sd128-ALL-1_min-v1jplv04.3_filesave4.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations\\PMEL_Arctic_2015_sd128-ALL-1_min-v1rssv04.0_filesave4.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations\\PMEL_Arctic_2016_sd126-ALL-1_min-v1jplv04.3_filesave4.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations\\PMEL_Arctic_2016_sd126-ALL-1_min-v1rssv04.0_filesave4.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations\\PMEL_Arctic_2016_sd128-ALL-1_min-v1jplv04.3_filesave4.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations\\PMEL_Arctic_2016_sd128-ALL-1_min-v1rssv04.0_filesave4.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations\\saildrone-gen_4-baja_2018-sd1002-20180411T

In [ ]:
for iname,name in enumerate(filenames):
    i = name.find('\\')
    fout = data_dir_out + name[i+1:-3]+'norep.nc' 
    ds = xr.open_dataset(name)
    ds.close()
    ds_tem2 = ds.where((ds.smap_SSS<50) & (ds.smap_SSS>1),drop=True)
    isv=0
    while len(ds_tem2.time)>1:
        i=0
        cond = ((ds_tem2.smap_name==ds_tem2.smap_name[i]) 
                & (ds_tem2.smap_ydim==ds_tem2.smap_ydim[i]) 
                & (ds_tem2.smap_xdim==ds_tem2.smap_xdim[i]))
        subset = ds_tem2.where(cond,drop=True)  #repeat obs
        ds_mn = subset.mean(keep_attrs=True,skipna=True)
        ds_mn['time'] = subset.time.mean()
        ds_mn = ds_mn.assign_coords({'ob':isv})
        if isv==0:
            ds_mn2 = ds_mn
        else:
            ds_mn2 = xr.concat([ds_mn2,ds_mn],dim='ob')
        isv = isv+1
        ds_tem2 = ds_tem2.where(~cond,drop=True)  #data with repeat obs removed
    ds_mn2.to_netcdf(fout)

In [ ]:
#ds= xr.open_dataset('F:/data/sat_data/smap/SSS/L2/JPL/V4.3/2015/136\SMAP_L2B_SSS_01530_20150516T034831_R16010_V4.3.h5')

In [4]:
file = data_dir_out + 'PMEL_Arctic_2015_sd126-ALL-1_min-v1jplv04.3_filesave4norep.nc'
ds = xr.open_dataset(file)
ds

<xarray.Dataset>
Dimensions:                       (ob: 334)
Coordinates:
  * ob                            (ob) int64 0 1 2 3 4 5 ... 329 330 331 332 333
Data variables:
    lat                           (ob) float64 ...
    lon                           (ob) float64 ...
    TEMP_AIR_MEAN                 (ob) float64 ...
    RH_MEAN                       (ob) float64 ...
    BARO_PRES_MEAN                (ob) float64 ...
    SAL_CTD_MEAN                  (ob) float64 ...
    TEMP_CTD_MEAN                 (ob) float64 ...
    TEMP_CTD_STDDEV               (ob) float64 ...
    CDOM_MEAN                     (ob) float64 ...
    CHLOR_MEAN                    (ob) float64 ...
    PAR_AIR_MEAN                  (ob) float64 ...
    GUST_WND_MEAN                 (ob) float64 ...
    UWND_MEAN                     (ob) float64 ...
    VWND_MEAN                     (ob) float64 ...
    SOG_MEAN                      (ob) float64 ...
    COG_MEAN                      (ob) float64 ...
    HDG_MEAN                      (ob) float64 ...
    ROLL_MEAN                     (ob) float64 ...
    PITCH_MEAN                    (ob) float64 ...
    WWND_MEAN                     (ob) float64 ...
    TEMP_RBR_MEAN                 (ob) float64 ...
    SAL_RBR_MEAN                  (ob) float64 ...
    TEMP_O2_RBR_MEAN              (ob) float64 ...
    CHLOR_WETLABS_MEAN            (ob) float64 ...
    WIND_MEASUREMENT_HEIGHT_MEAN  (ob) float64 ...
    deltaT                        (ob) float64 ...
    smap_SSS                      (ob) float32 ...
    smap_fland                    (ob) float32 ...
    smap_fice                     (ob) float32 ...
    smap_iqc_flag                 (ob) float64 ...
    smap_dist                     (ob) float64 ...
    smap_ydim                     (ob) float32 ...
    smap_xdim                     (ob) float32 ...
    smap_rev_number               (ob) float64 ...
    time                          (ob) datetime64[ns] ...
Attributes:
    history:      FERRET V7.3  19-Dec-17
    Conventions:  CF-1.6
    featureType:  trajectory
    project:      PMEL_Arctic_2015
    vehicle_id:   sd126